In [1]:
import requests
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import gamma
from sklearn import linear_model
import numpy as np

In [2]:
import c3aidatalake

In [29]:
# Returns datetime object of the given date string
# Compare dates with >, <, or ==
def timeFormat(s):
  return datetime.strptime(s, "%Y-%m-%dT%H:%M:%SZ")

# Returns float timestamp (seconds since 1 Jan 1970)
# Compare timestamps with >, <, or ==
def timestampFormat(s):
  return timeFormat(s).timestamp()

# How to convert column of date strings to these above formats:
# DFtimeFormat(dataframe name, string of column header which contains the time strings)
def DFtimeFormat(df, column_of_timestrings):
  df[column_of_timestrings] = df[column_of_timestrings].apply(timeFormat)
  return

In [31]:
# returns true if date2 comes after date1
def after(date1, date2):
    return True if date2>date1 else False

statemaskdates = {"California": 1, "Texas": 2, "NorthCarolina": 1.5} # for testing
surveystates = ["California","California","California", "Texas", "NorthCarolina", "NorthCarolina"] # for testing
surveydates = [0, 1, 2, 1.9, 2, 1.4] # for testing

# Pass in the dates for mask/stay-home order as STATEDATES, 
# the location column of surveydata as SURVEYSTATES and
# the starttime column of surveydata as SURVEYDATES
# returns a column AFTERS that you can directly append to your dataframe
def surveyAfterRule(statedates, surveystates, surveydates):
    afters = []
    for i in range(len(surveystates)):
        afters += [after(statedates[surveystates[i]], surveydates[i])]
    return afters

surveyAfterRule(statemaskdates, surveystates, surveydates) # for testing
        

[False, False, True, False, True, False]

###Todo (by 11/3):
- Quantify government policy categories (LocationPolicySummary)
    - Bigger values for more restrictive policies
- Convert economic data to usable format
- Produce/visualize distributions and general statistics for data
    - Visualize dates of latest policies for states


In [ ]:
policies = c3aidatalake.fetch(
    "locationpolicysummary", 
    {
        "spec" : {
            "limit" : -1
        }
    })

Generating numerical columns from statePolicies:

Sora's Dictionaries:
'mandatoryQuarantine',
       'nonEssentialBusiness', 'largeGatherings', 'schoolClosure',
       'restaurantLimit', 'barClosures', 'faceCoveringRequirement',
       'PrimaryElectionPostponement', 'emergencyDeclaration',


In [ ]:
{
    "tags": [
        "hide-input",
    ]
}
quantifyDicts = {}
quantifyDicts["easingOrder"] = {
    "Reopened" : 0, 
    "Proceeding with Reopening" : 1,
    "Paused" : 2, 
    "New Restrictions Imposed" : 3
}
quantifyDicts["stayAtHome"] = {
    "No Action" : 0,
    "Lifted" : 0,
    "Rolled Back to High Risk Groups" : 1,
    "New Stay at Home Order" : 2,
    "Statewide" : 2
}
quantifyDicts["mandatoryQuarantine"]  = {
    "No Action" : 0,
    "Lifted" : 0,
    "From Certain States (New)" : 1,
    "Rolled Back to Certain States" : 1,
    "From Certain States" : 1,
    "Rolled Back to International Travel" : 2,
    "All Travelers" : 3
}
quantifyDicts["nonEssentialBusiness"] = {
    "No Action" : 0,
    "All Non-Essential Businesses Permitted to Reopen" : 0,
    "Some Non-Essential Businesses Permitted to Reopen" : 1,
    "All Non-Essential Businesses Permitted to Reopen with Reduced Capacity" : 1,
    "Some Non-Essential Businesses Permitted to Reopen with Reduced Capacity" : 2,
    "New Business Closures or Limits" : 3
}
quantifyDicts["largeGatherings"] = {
    "Lifted" : 0,
    "No Action" : 0,
    "Expanded to New Limit Above 25" : 1,
    "New Limit on Large Gatherings in Place" : 1,
    "Expanded to New Limit of 25 or Fewer" : 2,
    ">10 People Prohibited" : 3,
    "All Gatherings Prohibited" : 4
}
quantifyDicts["schoolClosure"] = {
    "Rescinded" : 0,
    "Recommended Closure for School Year" : 1,
    "Recommended Closure" : 2,
    "Closed for School Year" : 3,
    "Closed" : 4
}
quantifyDicts["restaurantLimit"] = {
    "No Action" : 0,
    "Reopened to Dine-in Service" : 1,
    "Reopened to Dine-in Service with Capacity Limits" : 2,
    "New Service Limits" : 3,
    "Newly Closed to Dine-in Service" : 3
}
quantifyDicts["barClosures"] = {
    "Reopened" : 0,
    "New Service Limits" : 1,
    "Closed" : 2,
    "Newly Closed" : 2
}
quantifyDicts["faceCoveringRequirement"] = {
    "No" : 0,
    "Required for Certain Employees" : 1,
    "Allows Local Officals to Require for General Public" : 1,
    "Required for Certain Employees; Allows Local Officials to Require for General Public" : 1,
    "Required for General Public" : 2
}
quantifyDicts["PrimaryElectionPostponement"] = {
    "No" : 0,
    "Postponed" : 1,
    "Canceled" : 2
}
quantifyDicts["emergencyDeclaration"] = {
    "Yes" : 0
}
quantifyDicts["waiveTreatmentCost"]  = {
    "No Action" : 0,
    "State-Insurer Agreement" : 1,
    "State Requires" : 2
}  
quantifyDicts["freeVaccine"] = {
    "No Action" : 0,
    "State-Insurer Agreement" : 1,
    "State Requires" : 2
}
quantifyDicts["waiverOfPriorAuthorizationRequirements"] = {
    "No Action" : 0,
    "For COVID-19 Testing" : 1,
    "For COVID-19 Testing and Treatment" : 2
}
quantifyDicts["prescriptionRefill"] = {
    "No Action" : 0,
    "Expired" : 1,
    "State Requires" : 2
}
quantifyDicts["premiumPaymentGracePeriod"] = {
    "No Action" : 0,
    "Expired" : 1,
    "COVID-19 Diagnosis/Impacts Only" : 2,
    "Grace Period Extended for All Individual Policies" : 3,
    "All Policies" : 4
}
quantifyDicts["marketplaceSpecialEnrollmentPeriod"] = {
    "No" : 0,
    "Ended" : 1,
    "Active" : 2
}
quantifyDicts["section1135Waiver"] = {
    "Unapproved" : 0,
    "Approved" : 1
}
quantifyDicts["paidSickLeaves"] = {
    "No Action" : 0,
    "Proposed - March 2020" : 1,
    "Enacted" : 2
}
quantifyDicts["expandsAccesstoTelehealthServices"] = {
    "No" : 0,
    "Yes" : 1
}

In [ ]:
# dropping U.S. national policy
statePolicies = policies.drop(44)

In [ ]:
def mapperGenerator(colName):
    if colName in quantifyDicts.keys():
        def mapper(val):
            return quantifyDicts[colName][val]
    else:
        def mapper(val):
            return val
    return mapper

In [ ]:
# quantify policy
for col in statePolicies.columns:
    mper = mapperGenerator(col)
    statePolicies[col] = statePolicies[col].apply(mper)

In [ ]:
# read in stategdp info
# adding quarter 2 gdp change as a column to statepolicies
import re
stategdp = pd.read_excel(io = "qgdpstate1020_0.xlsx", index_col = 0, header = 1)
Q2GDPChange = []
for id in statePolicies["id"]:
    m = re.match(r"(.+)_UnitedStates_Policy$", id)
    Q2GDPChange.append(stategdp["2020Q2"][m[1]])
statePolicies.insert(len(statePolicies.columns), "Q2GDPChange", Q2GDPChange)

In [ ]:
X = statePolicies[['easingOrder', 'stayAtHome', 'mandatoryQuarantine', 'nonEssentialBusiness', 
                   'largeGatherings', 'schoolClosure', 'restaurantLimit', 'barClosures', 
                   'faceCoveringRequirement', 'paidSickLeaves']]
Y = statePolicies["Q2GDPChange"]
regr = linear_model.LinearRegression()
regr.fit(X, Y)

LinearRegression()

In [ ]:
xVars = ['easingOrder', 'stayAtHome', 'mandatoryQuarantine', 'nonEssentialBusiness', 'largeGatherings', 
         'schoolClosure', 'restaurantLimit', 'barClosures', 'faceCoveringRequirement', 'paidSickLeaves']
for i in range(len(xVars)):
    print("Coefficient of ", xVars[i], ": ", regr.coef_[i])

Coefficient of  easingOrder :  -0.9262141815009662
Coefficient of  stayAtHome :  -1.05923749331456
Coefficient of  mandatoryQuarantine :  -1.4291948106393213
Coefficient of  nonEssentialBusiness :  -0.35188468850207033
Coefficient of  largeGatherings :  -0.16549603747075345
Coefficient of  schoolClosure :  0.628377215294127
Coefficient of  restaurantLimit :  -0.5827684084902
Coefficient of  barClosures :  1.8856612498699614
Coefficient of  faceCoveringRequirement :  -1.0095592159160618
Coefficient of  paidSickLeaves :  1.1788408282659348


In [ ]:
print(regr.intercept_)

In [13]:
print(regr.score(X, Y))

0.25676549737962395
